In [18]:
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
from tqdm import tqdm
import time

# import cython module for allowinG c compilation and static binding to improve the performance
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


### Parameters

In [40]:
%%cython
import igraph as ig
import numpy as np

cdef double BETA = 0.5
cdef double MU = 0.5
cdef double P0 = 0.2
cdef int TMAX = 1000
cdef int TRANS = 900
cdef int NREP = 100

cdef int len_g = 100000

# initialize graph
g = ig.Graph.Barabasi(len_g, 5)

state = [np.random.binomial(1, P0) for _ in range(len_g)] # 1 means infected
old_state = state[:]

def update_node_state(g, state, old_state, vertex_id, mu=MU, beta=BETA):
    if old_state[vertex_id]:
        state[vertex_id] = np.random.binomial(1, mu)

    else:
        infected_vertices = sum([old_state[v] for v in g.neighbors(vertex_id)])
        state[vertex_id] = np.random.binomial(1, 1 - (1 - beta)**infected_vertices)

    return state[vertex_id]

In [41]:
infected_ratio_list = []
# Simulation loop
start_time = time.time()
for i in range(TMAX):
    if i%100 == 0:
        print("Iteration number: {}".format(i))
        start = time.time()
    old_state = state[:]

    infected_ratio_list.append(0)

    for j in range(len_g):
        infected_ratio_list[i] += update_node_state(g, state, old_state, j)

    infected_ratio_list[i] /= len_g
    
    if i%100 == 0:
        print("Iteration required time: {}\n".format(time.time()-start))
    
end_time = time.time()
print("\nTotal required time:{} seconds".format(end_time-start_time))


Iteration number: 0
Iteration required time: 0.014929533004760742

Iteration number: 100
Iteration required time: 0.0069811344146728516

Iteration number: 200
Iteration required time: 0.005944490432739258

Iteration number: 300
Iteration required time: 0.0

Iteration number: 400
Iteration required time: 0.0

Iteration number: 500
Iteration required time: 0.0

Iteration number: 600
Iteration required time: 0.006824493408203125

Iteration number: 700
Iteration required time: 0.005978822708129883

Iteration number: 800
Iteration required time: 0.0059969425201416016

Iteration number: 900
Iteration required time: 0.007992029190063477


Total required time:5.987779378890991 seconds


In [ ]:
#ig.plot(g)
plt.plot(infected_ratio_list)
plt.show()